In [54]:
import pandas as pd
import numpy as np
import datetime

# plot imports
import seaborn as sns
import matplotlib.pyplot as plt

# moddeling and forecasting
from xgboost import XGBRegressor


In [55]:
df = pd.read_csv("../measurements")

df.head()

,Variety,RGB_Image,Depth_Information,FreshWeightShoot,DryWeightShoot,Height,Diameter,LeafArea,ID,Week,Date
229,Satine,RGB_1.png,Depth_1.png,2.6,0.14,8.2,12.0,80.6,1,2021-W1,2021-01-04
362,Satine,RGB_2.png,Depth_2.png,2.1,0.10,8.5,8.2,67.4,2,2021-W1,2021-01-04
258,Satine,RGB_3.png,Depth_3.png,2.0,0.09,7.5,9.5,65.0,3,2021-W1,2021-01-04
263,Satine,RGB_4.png,Depth_4.png,2.7,0.13,6.0,10.6,78.7,4,2021-W1,2021-01-04
224,Lugano,RGB_5.png,Depth_5.png,2.3,0.11,8.3,9.8,73.9,5,2021-W1,2021-01-04


In [56]:
df["Date"].unique()

array(['2021-01-04', '2021-01-11', '2021-01-18', '2021-01-25',
       '2021-02-01', '2021-02-08', '2021-02-15'], dtype=object)

In [57]:
def prepare_dataframe(df):
    df['date_time'] = pd.to_datetime(df['Date'], format='%Y-%m-%d %H:%M:%S')
    # changed in dex to date
    df = df.set_index('date_time')

    # splice of unecessary colums
    df = df.loc(axis=1)["Variety", "FreshWeightShoot", "DryWeightShoot", "Height", "Diameter", "LeafArea","ID"]
    return df



In [58]:
df = prepare_dataframe(df)
df

,Variety,FreshWeightShoot,DryWeightShoot,Height,Diameter,LeafArea,ID
date_time,,,,,,,
2021-01-04,Satine,2.6,0.14,8.2,12.0,80.6,1
2021-01-04,Satine,2.1,0.10,8.5,8.2,67.4,2
2021-01-04,Satine,2.0,0.09,7.5,9.5,65.0,3
2021-01-04,Satine,2.7,0.13,6.0,10.6,78.7,4
2021-01-04,Lugano,2.3,0.11,8.3,9.8,73.9,5
...,...,...,...,...,...,...,...
2021-02-15,Satine,163.2,9.17,16.0,26.0,1858.3,387
2021-02-15,Satine,180.6,10.01,17.0,31.0,1805.6,388
2021-02-15,Satine,209.3,8.19,15.0,25.6,2257.1,389


In [69]:
def split_data(train_weeks, validation_weeks):
    # Raises exception when te requested amount of weeks is to big
    if train_weeks + validation_weeks > 6:
        raise Exception("To many weeks!")

    # Calculate datetime from weeks
    start_date = datetime.datetime(2021,1,4)
    end_train_date = start_date + datetime.timedelta(weeks=train_weeks -1 )
    end_validation_date = end_train_date + datetime.timedelta(weeks=validation_weeks)

    #format datetime to %Y-%m-%d
    end_train_date = end_train_date.strftime(format="%m-%d-%Y")
    end_validation_date = end_validation_date.strftime(format="%m-%d-%Y")

    data_train = df.loc[: end_train_date, :]
    data_val   = df.loc[end_train_date:end_validation_date, :]
    data_test  = df.loc[end_validation_date:, :]

    return data_train, data_val, data_test



In [70]:
data_train, data_val, data_test = split_data(4,2)

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 228 entries, 2021-01-04 to 2021-01-25
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Variety           228 non-null    object 
 1   FreshWeightShoot  228 non-null    float64
 2   DryWeightShoot    228 non-null    float64
 3   Height            228 non-null    float64
 4   Diameter          228 non-null    float64
 5   LeafArea          228 non-null    float64
 6   ID                228 non-null    int64  
dtypes: float64(5), int64(1), object(1)
memory usage: 14.2+ KB
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 181 entries, 2021-01-25 to 2021-02-08
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Variety           181 non-null    object 
 1   FreshWeightShoot  181 non-null    float64
 2   DryWeightShoot    181 non-null    float64
 3   Height            181 non-nu